In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import spacy
import matplotlib as mpl
import seaborn as sns
import warnings


warnings.filterwarnings('ignore')
plt.style.use('dark_background')
mpl.rcParams['axes.prop_cycle'] = plt.cycler(color=['blue'])

In [7]:
#get encoding of the data:
import chardet

with open('./train.csv' , 'rb') as f:
    result = chardet.detect(f.read(100000))
    # print(result)

df = pd.read_csv('./train.csv' , encoding = result['encoding'])
df.head()

,Item ID,Sentiment,SentimentText
0,1,0,"@RailMinIndia My PNR is 8348062961, I am in wa..."
1,2,0,@sureshpprabhu @RailMinIndia AC not working in...
2,3,0,@RailMinIndia I'm traveling to chennai by trai...
3,4,5,@RailMinIndia irctc is not responding at the t...
4,5,7,@DRMbhopal @RailMinIndia @sanjaygupta2012 @drm...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1366 entries, 0 to 1365
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Item ID        1366 non-null   int64 
 1   Sentiment      1366 non-null   int64 
 2   SentimentText  1366 non-null   object
dtypes: int64(2), object(1)
memory usage: 32.1+ KB
